# Kanapsack Problem databases 04:

* https://people.sc.fsu.edu/~jburkardt/datasets/knapsack_01/knapsack_01.html

# Atividade com nota:

* Avaliar o algoritmo Hill Climbing para as bases P01 a P07;
* Utilizar a função de aptidão knapsack do Mlrose;
* Apresentar a melhor solução encontrada e comparar com a melhor solução global disponível para a base de dados



In [2]:
!pip install mlrose

In [3]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import numpy as np

## Problem representation:
### 0/1 knapsack problem using a hill climbing algorithm for P04.
- The Problem: 
Given a set of items, each with a weight and a value, select a subset of the items to maximize the total value while keeping the total weight within a given capacity.

So the total_weight of any system configuration (any state) is constraint with a max_weight, I will call this max_weight W.

In [4]:
max_weight = 50
weights = [31, 10, 20, 19, 4, 3, 6]
values = [70, 20, 39, 37, 7, 5, 10]
state = np.array([1, 0, 0, 1, 0, 0, 0])
len(weights)

7

In [5]:
items = []
for i in range(len(weights)):
    items.append(('Item' + str(i), weights[i], values[i]))
    
items

[('Item0', 31, 70),
 ('Item1', 10, 20),
 ('Item2', 20, 39),
 ('Item3', 19, 37),
 ('Item4', 4, 7),
 ('Item5', 3, 5),
 ('Item6', 6, 10)]

This get_cost function is what I want to maximize. It's used to see which neighbor is better.

In [6]:
# Knapsack fitness function
def get_cost(state):
    total_value = sum(state[i] * values[i] for i in range(len(weights)))
    total_weight = sum(state[i] * weights[i] for i in range(len(weights)))
    if total_weight > max_weight:
        return 0
    return total_value

Which is nothing but this in an equation form:

$F(x_i) = \sum_{i=0}^{n-1} x_i v_i$,    if $\sum_{i=0}^{n-1} x_i w_i \leq W$ this is the constraint


Where $x_i$ represents a state vector $x = [x_0, x_1, \ldots, x_{n-1}]$. It denotes a number of copies of item i included in the knapsack.

In [7]:
# evaluate fitness function of given state
total_value = get_cost(state)
total_value

107

In [8]:
print("Fitness value: ", total_value)

Fitness value:  107


In [9]:
# custom fitness function object
fitness_cust = mlrose.CustomFitness(get_cost)

In [10]:
#create enviroment
problem = mlrose.DiscreteOpt(length = 7, fitness_fn = fitness_cust, maximize = True, max_val = 2)

### Hill Climb Method
- Main Idea: 
The algorithm iteratively improves the solution by exploring neighboring states and selecting the state with the highest value (accordingly with get_cost) until no better solution can be found or a maximum number of iterations is reached (100 below).
In other words, this algorithm maintain a single node and searches by moving to a neighboring node.

In [11]:
# Call the hill_climb function to solve the problem
best_state, best_fitness, curve = mlrose.hill_climb(problem, max_iters=100, restarts=0, init_state = state, curve=True)
best_state, best_fitness, curve

(array([1, 0, 0, 1, 0, 0, 0]), 107.0, array([], dtype=float64))

In [12]:
get_cost(best_state)

107

In [13]:
print(f"soluçãoHC: {best_state}")
print(f"Fitness Value: {best_fitness}")

soluçãoHC: [1 0 0 1 0 0 0]
Fitness Value: 107.0


### Introducing Random Restart 
Randomness should improve the optimization process value. Because for each iteration, it explores neighboring states by flipping the value (0 to 1 or 1 to 0) of a randomly selected item, and evaluates their costs and it keeps track of the best neighboring states with the highest cost.The current state is then updated to one of the best neighboring states. The process continues until no better neighbor is found or the maximum number of iterations is reached.

In [14]:
best_state, best_fitness, curve = mlrose.random_hill_climb(problem, max_iters=10, restarts=10, curve=True, random_state=100)
best_state, best_fitness, curve

(array([1, 0, 0, 1, 0, 0, 0]),
 107.0,
 array([ 88.,  88.,  88.,  88.,  88.,  88.,  88.,  88.,  88.,  88.,  46.,
         83.,  83.,  88.,  88.,  88.,  88.,  88.,  88.,  88.,   0.,   0.,
          0.,   0.,   0.,   0.,  74.,  74.,  74.,  74.,  79.,  79.,  79.,
         79.,  79.,  79.,  79.,  79.,  79.,  79.,  72.,  72.,  72.,  79.,
         79.,  79.,  79.,  79.,  79.,  79.,  86.,  86.,  93.,  93.,  93.,
         93.,  93.,  93.,  93.,  93., 107., 107., 107., 107., 107., 107.,
        107., 107., 107., 107.,  59.,  59.,  59.,  59.,  59.,  59.,  79.,
         79.,  79.,  79., 107., 107., 107., 107., 107., 107., 107., 107.,
        107., 107.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,  87.,  87.,  92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.]))

## Conclusion
Rather than just hill climb one time, we can hill climb multiple times and figure out what is the best one that I've been able to find.
So we've implemented a function for random restart that restarts some maximum number of times, and repeat, on that number of times, this hill-climb method 'random_hill_climb' and figure out what the cost is (for that state after randomly restart maximum times).
As we can see there is a lot of local maximums (72, 79, 105...),  the algorithm starts at 0 and runs through a lot of neighbors, always keeping track (even the lowest ones) of the best neighboring states with the highest cost. 107 is the global maximum for this dataset which was the given optimal state. 
So here the algorithm restarted multiple times and even though wasn't able to find the global maximum.

# Simulated Annealing
In order to find a global maximum or global minimum we may need to make a move that makes our situation worse. Because sometimes if we get stuck in a local max/min, I want to dislodge from that in order to find the global max/min.
 - Simulated Annealing is a great technique for that. 
Suppose some state-space, if I am in a current state and I'm looking for a global maximum and I'm trying to maximize the value of the state, Hill-Climbing would just take the state and look at the two neighbor ones, and always pick the one that is going to increase the value of the state. As said before "in order to find a global maximum/minimum we may need to make a move that makes our situation worse." such that later on we can find that global maximum.
Once found this global max state we probably don't wan't to be moving to states worse than current state. This is why this metaphor for annealing -> start making more random moves and, over time, start to make fewer of those random moves based on a particular temperature schedule.

In [15]:
# Define a schedule for simulated annealing (you can customize this schedule)
schedule = mlrose.ExpDecay(init_temp=100, exp_const=0.05, min_temp=1)

the idea is that this temperature is going to be higher early on and lower later on.
It is exponentially decaying according to the formula: $T(t) = T_{i}e^{-rt}$
where:
-  T(t) is the temperature at time t.
- $T(i)$ is the initial temperature at time t=0.
- r is the rate of exponential decay.
- t is the time variable.
- The exponential decay is represented by $e^{-rt}$.

For ex: You start off $T_{i}$=100, after 1s the temperature will be:

In [16]:
# Evaluate the temperature parameter at time t = 1 and exp_const=0.05
t1 = schedule.evaluate(1)
print(f"T(1) = {t1}")

T(1) = 95.1229424500714


In [17]:
# Call the simulated_annealing function to solve the problem
best_state, best_fitness, curve = mlrose.simulated_annealing(problem, schedule=schedule, max_attempts=10, max_iters=np.inf, init_state=None, curve=True, random_state=100)
best_state, best_fitness, curve

(array([1, 0, 0, 1, 0, 0, 0]),
 107.0,
 array([ 49.,  88.,  83.,  93.,  54.,   0.,  87.,  80.,   0., 107., 107.,
          0.,   0.,   0.,  75.,  75.,  95.,  90.,   0., 107., 107., 107.,
        107., 107., 107., 107., 107., 107., 107., 107.]))

so the goal of this whole process is as we begin our search to find the local/global max/min, we can dislodge ourselves if we get stuck at a local max/min in order to eventually make our way to exploring the part of the state space that is going to be the best. And then as the temperature decreases, start to make fewer of those random moves, that is, not moving aroung too much to get into another part of the state space.

The simulated annealing algorithm ran through numerous iterations to find an optimal combination of items to include in a knapsack. The curve data indicates how the solution evolved over iterations, with the algorithm converging on the optimal solution towards the end.

**Important**: In order for finding the global maximum, I had to **adjust the parameters** in the ExpDecay schedule and in the simulated_annealing function.

# Conclusion
- **Best State**: [1, 0, 0, 1, 0, 0, 0] This is an array representing the optimal solution found by the algorithm.
- **Best fitness**: 107 says that the combination of items in the best state yields a total value of 107.
- **Curve**: This is an array showing the fitness score at each iteration of the algorithm. It helps in understanding how the solution improved over time. The values fluctuate, indicating the exploration of different solutions. The repeated values of 107 towards the end suggest that the algorithm consistently found a solution with a fitness of 107, indicating it likely reached an optimal or near-optimal solution